In [7]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datefilter='2024-01-01'
datefilteryesterday='2024-12-31'

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()

def hist_data_dog(surface, sex, odds_low,odds_high,date):
    data=pd.read_sql_query(f"Select distinct * from results_{surface}_1 where date > '{date}' and date < '{datefilteryesterday}'",con=devengine)#.to_clipboard(index=False)
    data["Fav_Odds"] = data.Fav_Odds.astype(float)
    data["Dog_Odds"] = data.Dog_Odds.astype(float)
    data['Fav_Win'] = data['Winner']==data['Fav']
    data['DogWin'] = data['Winner']!=data['Fav']
    data=data[(data['Fav_Odds']>=odds_low)&(data['Fav_Odds']<=odds_high)]
    data=data[data['Sex']==sex]
    return data
hard_womens=hist_data_dog('HArd','Womens',1,2,datefilter)
hard_mens=hist_data_dog('HArd','Mens',1,2,datefilter)
clay_womens=hist_data_dog('Clay','Womens',1,2,datefilter)
clay_mens=hist_data_dog('Clay','Mens',1,2,datefilter)
temp=pd.concat([clay_womens,clay_mens]).sort_values(by='Date')
temp = temp.reset_index(drop=True)
temp['Fav_Loss'] = (temp['Fav_Win'] == 0).astype(int)
temp['Fav_Loss_Count'] = temp.groupby('Fav')['Fav_Loss'].cumsum()
temp['Dog_Win'] = (temp['Fav_Win'] == 0).astype(int)
temp['Dog_Win_Count'] = temp.groupby('Dog')['Dog_Win'].cumsum()
# Load the dataset
df = temp[(temp['Sex']!='Wobmens')&(temp['Fav_Odds']<2)&(temp['Fav_Odds']>1.4)]


# Feature selection and preprocessing
features = ['fav_percent', 'dog_percent', 'Elo_Fav_Elo', 'Elo_Dog_Elo', 
            'fav_rank', 'dog_rank', 'dog_rank_high', 'fav_rank_high','Fav_Top100','Dog_Top100','Dog_Win_Count','Fav_Loss_Count']
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Drop the 'Winner_Odds' column
df = df.drop(columns=['Winner_Odds'])


target = 'Fav_Win'

X = df[features]  # Features for training
y = df[target]  # Target variable

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42,shuffle=False)


# Split based on time (e.g., last 20% for testing)
split_index = int(len(X) * 0.95)
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Model training
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Prediction and evaluation
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Add predictions and actuals to the test set DataFrame
test_with_predictions = X_test.copy()
test_with_predictions['Actual'] = y_test
test_with_predictions['Predicted'] = y_pred

# Add Fav_Odds and Dog_Odds back to the test set for profit calculation
test_with_predictions = test_with_predictions.merge(
    df[['Date','Fav','Dog','Fav_Odds', 'Dog_Odds']], left_index=True, right_index=True
)

# Define the stake amount
stake = 100  # Example stake amount

# Profit calculation
def calculate_profit(row):
    if row['Predicted'] == 1:  # If the model predicts the favorite wins
        if row['Actual'] == 1:  # If the prediction is correct
            return stake * (row['Fav_Odds'] - 1)  # Profit based on favorite odds
        else:
            return -stake  # Loss of the stake
    else:  # If the model predicts the underdog wins
        if row['Actual'] == 0:  # If the prediction is correct
            return stake * (row['Dog_Odds'] - 1)  # Profit based on underdog odds
        else:
            return -stake  # Loss of the stake

test_with_predictions['Profit'] = test_with_predictions.apply(calculate_profit, axis=1)

# Save the output DataFrame to clipboard
fav_predictions=test_with_predictions[test_with_predictions['Predicted']==1].copy()
# Calculate overall profit
fav_profit = fav_predictions['Profit'].sum()
print(f"Fav Profit: ${fav_profit:.2f}",len(fav_predictions))

dog_predictions=test_with_predictions[test_with_predictions['Predicted']==0].copy()
# Calculate overall profit
dog_profit = dog_predictions['Profit'].sum()
print(f"Dog Profit: ${dog_profit:.2f}", len(dog_predictions))
dog_predictions.to_clipboard()
import numpy as np

# Define the odds bands (10c increments from 1.00 to 2.00)
bins = np.arange(1.0, 2.1, 0.1)  # Create bins: [1.0, 1.1, ..., 2.0]
labels = [f"{bins[i]:.1f} to {bins[i+1]:.1f}" for i in range(len(bins) - 1)]

# Create a new column for odds bands
dog_predictions['Fav_Odds_Band'] = pd.cut(dog_predictions['Fav_Odds'], bins=bins, labels=labels, right=False)

# Add a column to indicate if the prediction was correct
dog_predictions['Correct_Prediction'] = dog_predictions['Actual'] == dog_predictions['Predicted']

# Group by Fav_Odds_Band and calculate Profit, count of records, and winning percentage
grouped_dog = dog_predictions.groupby('Fav_Odds_Band').agg(
    Total_Profit=('Profit', 'sum'),
    Record_Count=('Profit', 'count'),
    Winning_Percentage=('Correct_Prediction', lambda x: (x.sum() / len(x) * 100) if len(x) > 0 else 0)
).reset_index()

# Print the results
print(grouped_dog)



import numpy as np

# Define the odds bands (10c increments from 1.00 to 2.00)
bins = np.arange(1.0, 2.1, 0.1)  # Create bins: [1.0, 1.1, ..., 2.0]
labels = [f"{bins[i]:.1f} to {bins[i+1]:.1f}" for i in range(len(bins) - 1)]

# Create a new column for odds bands
fav_predictions['Fav_Odds_Band'] = pd.cut(fav_predictions['Fav_Odds'], bins=bins, labels=labels, right=False)

# Add a column to indicate if the prediction was correct
fav_predictions['Correct_Prediction'] = fav_predictions['Actual'] == fav_predictions['Predicted']

# Group by Fav_Odds_Band and calculate Profit, count of records, and winning percentage
grouped_fav = fav_predictions.groupby('Fav_Odds_Band').agg(
    Total_Profit=('Profit', 'sum'),
    Record_Count=('Profit', 'count'),
    Winning_Percentage=('Correct_Prediction', lambda x: (x.sum() / len(x) * 100) if len(x) > 0 else 0)
).reset_index()

# Print the results
print(grouped_fav)


Accuracy: 0.6590909090909091
              precision    recall  f1-score   support

       False       0.53      0.73      0.62        66
        True       0.79      0.62      0.69       110

    accuracy                           0.66       176
   macro avg       0.66      0.67      0.65       176
weighted avg       0.69      0.66      0.66       176

Fav Profit: $2223.00 86
Dog Profit: $1691.00 90
  Fav_Odds_Band  Total_Profit  Record_Count  Winning_Percentage
0    1.0 to 1.1           0.0             0            0.000000
1    1.1 to 1.2           0.0             0            0.000000
2    1.2 to 1.3           0.0             0            0.000000
3    1.3 to 1.4           0.0             0            0.000000
4    1.4 to 1.5        1188.0            17           64.705882
5    1.5 to 1.6         -64.0            24           41.666667
6    1.6 to 1.7         -45.0            24           45.833333
7    1.7 to 1.8          90.0            17           52.941176
8    1.8 to 1.9     

In [8]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datefilter='2024-01-01'
datefilteryesterday='2025-01-30'

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()

def hist_data_dog(surface, sex, odds_low,odds_high,date):
    data=pd.read_sql_query(f"Select distinct * from results_{surface}_1 where date > '{date}' and date < '{datefilteryesterday}'",con=devengine)#.to_clipboard(index=False)
    data["Fav_Odds"] = data.Fav_Odds.astype(float)
    data["Dog_Odds"] = data.Dog_Odds.astype(float)
    data['Fav_Win'] = data['Winner']==data['Fav']
    data['DogWin'] = data['Winner']!=data['Fav']
    data=data[(data['Fav_Odds']>=odds_low)&(data['Fav_Odds']<=odds_high)]
    data=data[data['Sex']==sex]
    return data

hard_womens=hist_data_dog('HArd','Womens',1,2,datefilter)
hard_mens=hist_data_dog('HArd','Mens',1,2,datefilter)
clay_womens=hist_data_dog('Clay','Womens',1,2,datefilter)
clay_mens=hist_data_dog('Clay','Mens',1,2,datefilter)
temp=pd.concat([clay_womens,clay_mens]).sort_values(by='Date')
temp = temp.reset_index(drop=True)
temp['Fav_Loss'] = (temp['Fav_Win'] == 0).astype(int)
temp['Fav_Loss_Count'] = temp.groupby('Fav')['Fav_Loss'].cumsum()
temp['Dog_Win'] = (temp['Fav_Win'] == 0).astype(int)
temp['Dog_Win_Count'] = temp.groupby('Dog')['Dog_Win'].cumsum()

# Filter matches after 2024-12-31
today = temp[temp['Date'] > '2024-12-31'].copy()
today["Fav_Odds"] = today.Fav_Odds.astype(float)
today["Dog_Odds"] = today.Dog_Odds.astype(float)

# Apply odds filter
today = today[today['Fav_Odds'] > 1.4]

# Prepare features and make predictions
today_features = today[features]
today['Predicted'] = model.predict(today_features)
# Predict probabilities for today's features
probabilities = model.predict_proba(today_features)

# Extract the probability of the positive class (e.g., `Fav_Win` = 1)
today['Predicted_Prob'] = probabilities[:, 0]  # Column 1 corresponds to the positive class

# Add probabilities to the DataFrame
#today_features['Dog_Win_Probability'] = dog_win_probabilities

# Print the results
#print(today_features[['Fav_Win_Probability']])


# Filter for predicted underdog wins
today = today[today['Predicted'] == False]
today = today[today['Dog_Odds'] <33]
today = today[today['Dog_Odds'] >2]
today = today[today['Predicted_Prob']>0.6]

# Add a column to calculate profit/loss with a stake of 100 units
# Profit = (Dog_Odds - 1) * stake if the dog wins, otherwise -stake
stake = 100
today['Profit'] = today.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = today['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(today),len(today[today['Dog']==today['Winner']])/len(today))

# Save the data to clipboard
today.to_clipboard()

Total Profit: 364.0 10 0.6


In [13]:
temp.sort_values(by='Fav_Loss_Count',ascending=False)
temp.sort_values(by='Dog_Win_Count',ascending=False)

,Date,Winner_Odds,Winner,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Games,Dog,Dog_Games,...,Fav_Top100,Dog_Top100,fav_rank_high,dog_rank_high,Fav_Win,DogWin,Fav_Loss,Fav_Loss_Count,Dog_Win,Dog_Win_Count
333,2024-03-12,1.91,Ignacio Buse,1.83,1.91,Alexander Weis,Alexander Weis,11,Ignacio Buse,10,...,0.0,0.0,285.0,438.0,False,True,1,2,1,1
671,2024-04-02,2.04,Ignacio Buse,1.69,2.04,Pol Martin Tiffon,Ignacio Buse,10,Ignacio Buse,11,...,0.0,0.0,235.0,437.0,False,True,1,1,1,2
3523,2024-07-11,2.15,Ignacio Buse,1.65,2.15,Vit Kopriva,Vit Kopriva,12,Ignacio Buse,10,...,0.5,0.0,111.0,312.0,False,True,1,6,1,3
3579,2024-07-13,4.11,Ignacio Buse,1.21,4.11,Hamad Medjedovic,Hamad Medjedovic,10,Ignacio Buse,10,...,0.6,0.0,102.0,312.0,False,True,1,3,1,4
4554,2024-09-06,2.05,Ignacio Buse,1.72,2.05,Daniel Elahi Galan,Daniel Elahi Galan,13,Ignacio Buse,10,...,0.2,0.0,56.0,212.0,False,True,1,4,1,5
257,2024-02-25,1.26,Francisco Comesana,1.26,3.69,Francisco Comesana,Francisco Comesana,10,Ignacio Buse,9,...,0.3,0.0,117.0,438.0,True,False,0,3,0,0
359,2024-03-14,1.19,Camilo Ugo Carabelli,1.19,4.49,Camilo Ugo Carabelli,Camilo Ugo Carabelli,10,Ignacio Buse,10,...,0.7,0.0,96.0,438.0,True,False,0,2,0,1
667,2024-04-02,1.57,Valentin Royer,1.57,2.31,Valentin Royer,Valentin Royer,10,Ignacio Buse,10,...,1.0,0.0,250.0,437.0,True,False,0,0,0,1
948,2024-04-09,1.49,Leandro Riedi,1.49,2.51,Leandro Riedi,Leandro Riedi,7,Ignacio Buse,10,...,0.7,0.0,126.0,396.0,True,False,0,0,0,2
1391,2024-04-23,1.71,Carlos Taberner,1.71,2.06,Carlos Taberner,Carlos Taberner,11,Ignacio Buse,10,...,0.0,0.0,85.0,396.0,True,False,0,3,0,2
